# Quantum Molecular Dynamics #

“*In fact, the mere act of opening the box will determine the state of the
cat, although in this case there were three determinate states the cat
could be in: these being* **Alive**, **Dead**, *and* **Bloody Furious**.”
― Terry Pratchett, "Lords and Ladies" 

## Problem Description ##

TODO

## Mathematical Foundations ##

### Calculation of Eigenvalues ###

The goal of this work is to calculate distance of two alpha carbon atoms in the same segment or among two different segments. To this end, we need to calculate the lowest eigenvalue in a input matrix $H$, which is built in the following way:

Let $A$, $B$ be two segments with $n$ points. We define $H$ as:

$H = \begin{pmatrix}
        0^{n \times n} & D(A,B) \\
        D(A,B)^{T} & 0^{n \times n} \\
     \end{pmatrix}$

Such that each $D_{AB}$ cell contains the **squared euclidean distance** between each component of $A$ and $B$, namely, $d(a_i,b_i) = \sqrt{(a_x - b_x)^2 + (a_y - b_y)^2}$.

Calculation of eigenvalues in quantum machine is defined as follows: given a Hermitian matrix $H$ and a minimum eigenvalue $\lambda_{min}$, associated to a quantum eigenstate $|\Psi_{min}\rangle$, we want to find a $\lambda_{\Theta}$ such that

$\lambda_{min} \leq \lambda_{\Theta} \equiv \langle \Psi(\Theta)|H|\Psi(\Theta)\rangle $

Where $\psi(\Theta)$ is the eigenstate associated to $\lambda_{\Theta}$ and $\Theta$ is the parameter of quantum circuit, which is used to minimize $\lambda_{\Theta}$.

**Th**: Input matrix $H$ is **Hermitian**.

**Proof**: Let $H \in \mathbb{C}^{n} \times \mathbb{C}^{n}$, with $n \in \mathbb{N}$. $H$ is **Hermitian** $\iff H = H^{\dagger}$.

$H^{\dagger}$ is the **adjunct** of $H$, i.e., the **transpose complex conjugate** of $H$, meaning that $H_{ij} = \overline{H_{ji}}$. 

For $0$ cells, complex conjugate is 0. For cells of $D_{AB}$, we know that $d$ is defined as an application $\mathbb{V} \times \mathbb{V} \mapsto \mathbb{R}$. Since for each $x \in \mathbb{R}, \overline{x} = x$, we have that $H = \overline{H}$.

Now, what's left to prove is that $H = H^{T}$. To prove it, we calculate the transpose of $H$ as follows:

$H^{T} = \begin{pmatrix}
           0 & \dots & 0 & D(A,B)_{0,0} & \dots & D(A,B)_{0,n} \\
           \vdots & \ddots & \vdots & \vdots & \ddots & \vdots \\
           0 & \dots & 0 & D(A,B)_{n,0} & \dots & D(A,B)_{n,n}  \\
           D(A,B)_{0,0} & \dots & D(A,B)_{n,0} & 0 & \dots & 0\\
           \vdots & \ddots & \vdots & \vdots & \ddots & \vdots \\
           D(A,B)_{0,n} & \dots & D(A,B)_{n,n} & 0 & \dots & 0\\
        \end{pmatrix}^{T} = \begin{pmatrix}
           0 & \dots & 0 & D(A,B)_{0,0} & \dots & D(A,B)_{0,n} \\
           \vdots & \ddots & \vdots & \vdots & \ddots & \vdots \\
           0 & \dots & 0 & D(A,B)_{n,0} & \dots & D(A,B)_{n,n}  \\
           D(A,B)_{0,0} & \dots & D(A,B)_{n,0} & 0 & \dots & 0\\
           \vdots & \ddots & \vdots & \vdots & \ddots & \vdots \\
           D(A,B)_{0,n} & \dots & D(A,B)_{n,n} & 0 & \dots & 0\\
        \end{pmatrix}$

Since $H_{i,j}^{T} = H_{j,i}.~\mathbf{\square}$

The main advantage of $H$ being Hermitian is that all the eigenvalues are real, therefore are suitable representations for the Hamiltonians of quantum systems.